In [ ]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from IPython.display import Javascript
import pandas as pd
import csv
import os

---

<center><h1><b>MarkUp</b></h1></center>

---

- Udemy site is using in technology called "dynamic JavaScript" which means the website is not fully loadded
- when the user enters the course he would like to get.
- BeautifulSoup could not handle the "dynamic JavaScript" properly and therefore needed to be used the package 
- of Selenium
- The code is organized in such a way:
- run_web_driver ==> The code which runs the chrome driver with all the configurations
- get_url_content ==> get the html content
- create_soup_object ==> create BeautifulSoup object
- get_total_number_of_pages ==> when going over name of course in the search then you need to find out how many pages do you have. for example: for python course there will be 500 pages so we would glad to know how many pages to extract their links
- extract_links ==> after getting the html going to the right element and build from it the URLS of the courses
- write_to_file ==> write the URLS to UDEMY_URLS file in order to make API requests afterwards
- forward_to_next_page ==> forward to next page in the udemy site.. happeens when there is a success/failure in getting the URL name

In [ ]:
#global variables
global page_num
page_num = 1
global number_pages
number_pages = -1
global num_of_tries
num_of_tries = 0

###################################################################
####################### courses names for query ###################
###################################################################
#####[python, css, react, javascript, java, c#, c++, testing]######
###################################################################
global courses_names
courses_names = ['python', 'css', 'react', 'javascript', 'java', 'c%23', 'c%2B%2B', 'testing']
global course_index
course_index = 0

def run_web_driver():
    opts = Options()
    os.environ['WDM_LOG_LEVEL'] = '0'
    opts.add_argument("start-maximized")
    opts.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
#     driver = webdriver.Chrome(ChromeDriverManager().install())
    agent = driver.execute_script("return navigator.userAgent")
    opts.add_argument("user-agent=" + agent)
    driver = webdriver.Chrome(chrome_options = opts)
    driver.maximize_window()
    time.sleep(8)
    get_url_content(driver)

def get_url_content(driver):
    global page_num
    global courses_names
    global course_index

    url = f'https://www.udemy.com/courses/search/?p={page_num}&q={courses_names[course_index]}&src=ukw'
    try:
        driver.get(url)
        time.sleep(10)
        create_soup_object(driver)
    except:
        print("An exception occurred during driver.get url")
        forward_to_next_page()

def create_soup_object(driver):
    global number_pages
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    if soup == None:
        driver.close()
        run_web_driver()
    driver.close()
    
    if number_pages == -1: 
        get_total_number_of_pages(soup)
    else:
        extract_links(soup)

def get_total_number_of_pages(soup):
    global number_pages

    number_pages_soup = soup.find('span', attrs={'class':'udlite-heading-sm pagination--page--1H0A2'})
    if number_pages_soup == None:
        run_web_driver()
        
    number_pages = int(number_pages_soup.text)
    extract_links(soup)

def extract_links(soup):
    global num_of_tries

    links = [item['href'] for item in soup.select('body div h3 a')]
    if not links:
        num_of_tries = num_of_tries + 1
        if num_of_tries == 3:
            print('error during links extraction')
            forward_to_next_page()
        else:
            run_web_driver()
    else:
        num_of_tries = 0
        write_to_file(links)

def write_to_file(links):
    with open('Udemy_URLS.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        for link in links:            
            writer.writerow({'www.udemy.com' + link})
    forward_to_next_page()

def forward_to_next_page():
    global page_num
    global number_pages
    global num_of_tries
    global courses_names
    global course_index

    if page_num <= number_pages:
        page_num = page_num + 1
        run_web_driver()
    else:
        page_num = 1
        number_pages = -1
        num_of_tries = 0
        course_index = course_index + 1
        if(course_index < len(courses_names)):
            run_web_driver()
        else:
            print('finished!')
            
            
#start of the program
run_web_driver()

D:\Anaconda\Anaconda3\envs\mlcourse\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options


error during links extraction
error during links extraction
An exception occurred during driver.get url
